In [18]:

# Load your stock price data into a DataFrame
# Replace 'your_data.csv' with the path to your data file
data = pd.read_excel('final_data_with_covid_same.xlsx')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)


In [19]:
print(data.head())

             EPS  Book Value   COF  Dividend  inflation  \
Date                                                      
2018-04-27  46.7       210.0  4.63      0.48        6.0   
2018-04-28  46.7       210.0  4.63      0.48        6.0   
2018-04-29  46.7       210.0  4.63      0.48        6.0   
2018-04-30  46.7       210.0  4.63      0.48        6.0   
2018-05-01  46.7       210.0  4.63      0.48        6.0   

            Remittance Growth Rate(%)    Open    High     Low   Close  Volume  \
Date                                                                            
2018-04-27                        7.0  1050.0  1050.0  1020.0  1030.0   13380   
2018-04-28                        7.0  1050.0  1050.0  1020.0  1030.0   13380   
2018-04-29                        7.0  1015.0  1020.0  1002.0  1015.0    7378   
2018-04-30                        7.0  1015.0  1020.0  1002.0  1015.0    7378   
2018-05-01                        7.0  1015.0  1020.0  1002.0  1015.0    7378   

             MA_10

In [20]:
data.columns

Index(['EPS', 'Book Value', 'COF', 'Dividend', 'inflation',
       'Remittance Growth Rate(%)', 'Open', 'High', 'Low', 'Close', 'Volume',
       'MA_10', 'Bollinger_Middle', 'Bollinger_Upper', 'Bollinger_Lower',
       'MACD', 'Signal_Line', 'MA_30', 'Compound Sensitivity'],
      dtype='object')

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Create a simple time series (list of numbers)
time_series = [10, 12, 15, 18, 20, 22, 25, 28, 30, 32]

# Define the number of splits for time series cross-validation
n_splits = 3

# Initialize variables to store MSE values and predictions
mse_values = []
predictions = []

# Initialize a TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=n_splits)

# Loop through the time series cross-validation splits
for train_index, test_index in tscv.split(time_series):
    train_data = [time_series[i] for i in train_index]
    test_data = [time_series[i] for i in test_index]
    print(train_data, test_data, sep="tt")
    


[10, 12, 15, 18]tt[20, 22]
[10, 12, 15, 18, 20, 22]tt[25, 28]
[10, 12, 15, 18, 20, 22, 25, 28]tt[30, 32]


In [28]:
from statsmodels.tsa.arima.model import ARIMA
help(ARIMA)

Help on class ARIMA in module statsmodels.tsa.arima.model:

class ARIMA(statsmodels.tsa.statespace.sarimax.SARIMAX)
 |  ARIMA(endog, exog=None, order=(0, 0, 0), seasonal_order=(0, 0, 0, 0), trend=None, enforce_stationarity=True, enforce_invertibility=True, concentrate_scale=False, trend_offset=1, dates=None, freq=None, missing='none', validate_specification=True)
 |  
 |  Autoregressive Integrated Moving Average (ARIMA) model, and extensions
 |  
 |  This model is the basic interface for ARIMA-type models, including those
 |  with exogenous regressors and those with seasonal components. The most
 |  general form of the model is SARIMAX(p, d, q)x(P, D, Q, s). It also allows
 |  all specialized cases, including
 |  
 |  - autoregressive models: AR(p)
 |  - moving average models: MA(q)
 |  - mixed autoregressive moving average models: ARMA(p, q)
 |  - integration models: ARIMA(p, d, q)
 |  - seasonal models: SARIMA(P, D, Q, s)
 |  - regression with errors that follow one of the above ARIM

In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


# Define the range for d values
d_values = range(1, 100)

# Initialize variables to store the best model and its MSE
best_model = None
best_mse = float('inf')
best_d = None

# Define your time series cross-validation method
tscv = TimeSeriesSplit(n_splits=100)  # You can adjust the number of splits

# Define the columns you want to use as features
feature_cols = ['EPS', 'Book Value', 'COF', 'Dividend', 'inflation', 'Remittance Growth Rate(%)',
                'Open', 'High', 'Low', 'Close', 'Volume', 'MA_10', 'Bollinger_Middle',
                'Bollinger_Upper', 'Bollinger_Lower', 'MACD', 'Signal_Line', 'MA_30', 'Compound Sensitivity']

# Define the target variable
target_col = 'Close'

# Loop through different d values
for d in d_values:
    mse_values = []  # To store MSE values for each fold
    
    for train_index, test_index in tscv.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]
        
        # Fit an ARIMA model with differencing order d
        model = ARIMA(train_data[target_col], order=(0, d, 0))
        model_fit = model.fit()
        
        # Make predictions
        predictions = model_fit.forecast(steps=len(test_data))
        
        # Calculate MSE and store it
        mse = mean_squared_error(test_data[target_col], predictions)
        mse_values.append(mse)
    
    # Calculate the average MSE for the current d value
    avg_mse = np.mean(mse_values)
    
    # Update the best model if the current d value results in a lower MSE
    if avg_mse < best_mse:
        best_model = model_fit
        best_mse = avg_mse
        best_d = d

print(f'Best ARIMA Model: ARIMA(0, {best_d}, 0)')
print(f'Best MSE: {best_mse}')


/home/sailesh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/sailesh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/sailesh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/sailesh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/sailesh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferre

KeyboardInterrupt: 